In [1]:
import pandas as pd

## **Ejercicio 1**
> Realice una consulta que muestre el porcentaje de avance de créditos de cada alumno con una bandera que clasifique a los alumnos de la siguiente manera: 
- Realice una consulta que muestre las tres calificaciones mas altas obtenidas en cada asignatura.

- En la misma salida muestre el promedio de cada materia y una bandera que compare esta 
calificación con la obtenida por los alumnos en la primera consulta donde ‘1’ significa que la 
calificación fue mas alta que el promedio y ‘0’ que fue más baja. 
- Con el uso de Python guarde en un CSV a los alumnos que tienen una calificación mayor al 
promedio.
---

In [2]:
df_asignaturas = pd.read_csv('Asignaturas.csv', sep=',')
df_estudiantes = pd.read_csv('estudiantes.csv', sep=',')
df_inscripciones = pd.read_csv('Inscripcion.csv', sep=',')

In [3]:
df_asignaturas.head()

,asignatura_id,nombre,Semestre
0,2001,Algebra,1
1,2002,Calculo Diferencial,1
2,2003,Geometria Analitica,1
3,2004,Quimica,1
4,2005,Cultura y Comunicacion,1


In [4]:
df_estudiantes.head()

,numero_cuenta,nombre
0,312232188,Adelina Garcia Sanchez
1,312232189,Juan Jose Ballesteros Aguirre
2,312232190,Eugenio Galan Giron
3,312232191,Lupita de Esteban
4,312232192,Paulina Arino Portillo


In [5]:
df_inscripciones.head()

,curso_id,calificacion,num_cuenta,asignatura_id
0,12001,8,312232188,2001
1,12002,7,312232189,2001
2,12003,6,312232190,2001
3,12004,9,312232191,2001
4,12005,7,312232192,2001


In [6]:
# unir las tablas
df_completo = (df_inscripciones
               .merge(df_estudiantes, left_on='num_cuenta', right_on='numero_cuenta', how='inner')
               .merge(df_asignaturas, on='asignatura_id', how='inner')
               .rename(columns={'nombre_x': 'nombre_estudiante', 'nombre_y': 'nombre_materia'}))
df_completo.head()

,curso_id,calificacion,num_cuenta,asignatura_id,numero_cuenta,nombre_estudiante,nombre_materia,Semestre
0,12001,8,312232188,2001,312232188,Adelina Garcia Sanchez,Algebra,1
1,12002,7,312232189,2001,312232189,Juan Jose Ballesteros Aguirre,Algebra,1
2,12003,6,312232190,2001,312232190,Eugenio Galan Giron,Algebra,1
3,12004,9,312232191,2001,312232191,Lupita de Esteban,Algebra,1
4,12005,7,312232192,2001,312232192,Paulina Arino Portillo,Algebra,1


In [7]:
# Promedio por materia
promedio_por_materia = df_completo.groupby(['asignatura_id', 'nombre_materia'])['calificacion'].mean().round(2).reset_index()
promedio_por_materia.columns = ['asignatura_id', 'nombre_materia', 'promedio']

print("Promedio por materia:")
promedio_por_materia.head()

Promedio por materia:


,asignatura_id,nombre_materia,promedio
0,2001,Algebra,7.68
1,2002,Calculo Diferencial,7.52
2,2003,Geometria Analitica,7.35
3,2004,Quimica,7.90
4,2005,Cultura y Comunicacion,7.68


In [8]:
# Top 3 calificaciones más altas por asignatura
top_3 = (df_completo
                        .groupby(['asignatura_id', 'nombre_materia'])
                        .apply(lambda x: x.nlargest(3, 'calificacion'))
                        .reset_index(drop=True))
print("Top 3 calificaciones por materia:")
top_3.head(9)

Top 3 calificaciones por materia:


,curso_id,calificacion,num_cuenta,asignatura_id,numero_cuenta,nombre_estudiante,nombre_materia,Semestre
0,12018,10,312232205,2001,312232205,Karina Matamoroz,Algebra,1
1,12024,10,312232211,2001,312232211,Javier Cruz Rojas,Algebra,1
2,12025,10,312232212,2001,312232212,Fanny Rojas Tello,Algebra,1
3,12036,10,312232192,2002,312232192,Paulina Arino Portillo,Calculo Diferencial,1
4,12040,10,312232196,2002,312232196,Anastasia Cruz Salazar,Calculo Diferencial,1
5,12044,10,312232200,2002,312232200,Kike Vicens Bou,Calculo Diferencial,1
6,12076,10,312232201,2003,312232201,Juliana Jodar,Geometria Analitica,1
7,12079,10,312232204,2003,312232204,Vania Castellanos Juarez,Geometria Analitica,1
8,12082,10,312232207,2003,312232207,Jorge Jimenez Palma,Geometria Analitica,1


In [9]:
# Resultado final con banderas
resultado_final = (top_3
                   .merge(promedio_por_materia, on=['asignatura_id', 'nombre_materia'], how='left'))

# Agregar bandera de comparación con promedio
resultado_final['bandera'] = (resultado_final['calificacion'] > resultado_final['promedio']).astype(int)

# Ordenar las columnas
columnas_ordenadas = ['nombre_estudiante', 'nombre_materia', 'calificacion', 'promedio', 'bandera']
resultado_final = resultado_final[columnas_ordenadas]

print("Resultado final - Top 3 calificaciones por materia con bandera:")
resultado_final

Resultado final - Top 3 calificaciones por materia con bandera:


,nombre_estudiante,nombre_materia,calificacion,promedio,bandera
0,Karina Matamoroz,Algebra,10,7.68,1
1,Javier Cruz Rojas,Algebra,10,7.68,1
2,Fanny Rojas Tello,Algebra,10,7.68,1
3,Paulina Arino Portillo,Calculo Diferencial,10,7.52,1
4,Anastasia Cruz Salazar,Calculo Diferencial,10,7.52,1
...,...,...,...,...,...
116,Rosalva Vargas,Bases de Datos Avanzadas,5,7.00,0
117,Rosalva Vargas,Bases de Datos Espaciales,9,7.33,1
118,Nico Verdugo Blanes,Bases de Datos Espaciales,7,7.33,0
119,Kike Vicens Bou,Bases de Datos Espaciales,6,7.33,0


In [10]:
# Estudiantes con calificación mayor al promedio
estudiantes_sobre_promedio = df_completo.merge(promedio_por_materia, on=['asignatura_id', 'nombre_materia'], how='left')
estudiantes_sobre_promedio = estudiantes_sobre_promedio[estudiantes_sobre_promedio['calificacion'] > estudiantes_sobre_promedio['promedio']]

 # Bandera de comparación con promedio
estudiantes_sobre_promedio['bandera'] = (estudiantes_sobre_promedio['calificacion'] > estudiantes_sobre_promedio['promedio']).astype(int)

# Columnas relevantes
estudiantes_csv = estudiantes_sobre_promedio[['numero_cuenta', 'nombre_estudiante', 'nombre_materia', 'calificacion', 'promedio','bandera']].copy()

# Guardar en CSV
estudiantes_csv.to_csv('estudiantes_sobre_promedio.csv', index=False, encoding='utf-8')
estudiantes_csv.head(10)

,numero_cuenta,nombre_estudiante,nombre_materia,calificacion,promedio,bandera
0,312232188,Adelina Garcia Sanchez,Algebra,8,7.68,1
3,312232191,Lupita de Esteban,Algebra,9,7.68,1
5,312232193,Maria Barriga Sandoval,Algebra,9,7.68,1
6,312232194,Wilfredo Corominas Zabala,Algebra,8,7.68,1
10,312232198,Nico Verdugo Blanes,Algebra,8,7.68,1
13,312232201,Juliana Jodar,Algebra,8,7.68,1
14,312232202,Alberto Herrera Cortez,Algebra,9,7.68,1
15,312232203,Diego Luz Alvares,Algebra,9,7.68,1
17,312232205,Karina Matamoroz,Algebra,10,7.68,1
18,312232206,Alexdr Nikolenko,Algebra,8,7.68,1


## **Ejercicio 2**
> Realice una consulta que muestre el porcentaje de avance de créditos de cada alumno con una bandera que clasifique a los alumnos de la siguiente manera: 
- Bandera ‘5’: Alumnos que tienen el 100%  de créditos  
- Bandera ‘4’: Alumnos que tienen entre >=80% y <100% de créditos  
- Bandera ‘3’: Alumnos que tienen entre >=65% y <80% de créditos 
- Bandera ‘2’: Alumnos que tienen entre >=40% y <65% de créditos 
- Bandera ‘1’: Alumnos que tienen entre >=0% y <40% de créditos
---

In [11]:
# Total de materias en el programa
total_materias_programa = df_asignaturas['asignatura_id'].nunique()
print(f"Total de materias en el programa: {total_materias_programa}")

# Materias cursadas por cada estudiante
materias_por_estudiante = (df_completo
                          .groupby(['numero_cuenta', 'nombre_estudiante'])
                          .agg({
                              'asignatura_id': 'nunique',  # Materias cursadas
                              'calificacion': 'count'      # Total de inscripciones
                          })
                          .rename(columns={
                              'asignatura_id': 'total_materias_cursadas',
                              'calificacion': 'total_inscripciones'
                          })
                          .reset_index())

print("Primeros registros de materias x estudiante:")
materias_por_estudiante.head()

Total de materias en el programa: 51
Primeros registros de materias x estudiante:


,numero_cuenta,nombre_estudiante,total_materias_cursadas,total_inscripciones
0,312232188,Adelina Garcia Sanchez,10,10
1,312232189,Juan Jose Ballesteros Aguirre,10,10
2,312232190,Eugenio Galan Giron,15,15
3,312232191,Lupita de Esteban,26,26
4,312232192,Paulina Arino Portillo,26,26


In [12]:
# Calcular porcentaje de avance
avance_estudiantes = materias_por_estudiante.copy()
avance_estudiantes['total_materias'] = total_materias_programa
avance_estudiantes['porcentaje_avance'] = (avance_estudiantes['total_materias_cursadas'] / total_materias_programa * 100).round(2)

# Banderas de clasificación
def asignar_bandera(porcentaje):
    if porcentaje == 100:
        return 5
    elif porcentaje >= 80:
        return 4
    elif porcentaje >= 65:
        return 3
    elif porcentaje >= 40:
        return 2
    else:
        return 1

avance_estudiantes['bandera_clasificacion'] = avance_estudiantes['porcentaje_avance'].apply(asignar_bandera)

# Reordenar columnas
columnas_finales = ['numero_cuenta', 'nombre_estudiante', 'total_materias_cursadas', 'total_materias', 'porcentaje_avance', 'bandera_clasificacion']
avance_estudiantes = avance_estudiantes[columnas_finales]

print("Avance de créditos por estudiante:")
avance_estudiantes.head(10)

Avance de créditos por estudiante:


,numero_cuenta,nombre_estudiante,total_materias_cursadas,total_materias,porcentaje_avance,bandera_clasificacion
0,312232188,Adelina Garcia Sanchez,10,51,19.61,1
1,312232189,Juan Jose Ballesteros Aguirre,10,51,19.61,1
2,312232190,Eugenio Galan Giron,15,51,29.41,1
3,312232191,Lupita de Esteban,26,51,50.98,2
4,312232192,Paulina Arino Portillo,26,51,50.98,2
5,312232193,Maria Barriga Sandoval,32,51,62.75,2
6,312232194,Wilfredo Corominas Zabala,32,51,62.75,2
7,312232195,Dalila Sanmiguel Pedrosa,37,51,72.55,3
8,312232196,Anastasia Cruz Salazar,37,51,72.55,3
9,312232197,Calisto Bolanos Bravo,15,51,29.41,1


In [13]:
# Obtener TOP 10 y BOTTOM 10 de alumnos por avance de créditos

# TOP 10 con mayor avance
top_10_estudiantes = (avance_estudiantes
                     .nlargest(10, 'porcentaje_avance')
                     .reset_index(drop=True))

# BOTTOM 10 de menor avance
bottom_10_estudiantes = (avance_estudiantes
                        .nsmallest(10, 'porcentaje_avance')
                        .reset_index(drop=True))


In [14]:
# Renombrar columnas
top_10_estudiantes_final = top_10_estudiantes.copy()
top_10_estudiantes_final = top_10_estudiantes_final.rename(columns={'nombre_estudiante': 'nombre_alumno'})

bottom_10_estudiantes_final = bottom_10_estudiantes.copy()
bottom_10_estudiantes_final = bottom_10_estudiantes_final.rename(columns={'nombre_estudiante': 'nombre_alumno'})

# Preparar columnas
columnas_csv = ['nombre_alumno', 'total_materias_cursadas', 'total_materias', 'porcentaje_avance', 'bandera_clasificacion']

# CSV con TOP 10 estudiantes
top_10_nombres = top_10_estudiantes_final[columnas_csv].copy()
top_10_nombres.to_csv('top_10_mayor_avance.csv', index=False, encoding='utf-8')

# CSV con BOTTOM 10 estudiantes
bottom_10_nombres = bottom_10_estudiantes_final[columnas_csv].copy()
bottom_10_nombres.to_csv('bottom_10_menor_avance.csv', index=False, encoding='utf-8')




## **Pregunta 3**
> Explique cual sería el procedimiento para cargar los archivos CSV generados en el ejercicio anterior en una base de datos mediante el uso de alguna herramienta ETL que usted conozca. 

---

Flujo del Proceso con Airflow:

- Como parte inicial, creación de un **Sensor Operator**, FileSensor, para esperar a que los archivos aparezcan en la carpeta destino.

- Leer y procesar los archivos, usando un PythonOperator para leer cada CSV con pandas por ejemplo, validar e incluso transformar los datos en caso de ser necesario.

- Cargar a base de datos MySQL, usando u MysqlOperator o un PythonOperator que utilice SQLAlchemy/pymysql para insertar los datos en la base de datos de MySQL, mapeando as columnas del CSV a las tablas correspondientes.

- Finalmente verificar los datos, agregando otro PythonOperator para consultar la base y verificar que los registros fueron insertados correctamente.

---

Podemos programar el DAG para que corra automáticamente cada que haya nuevos archivos o cada determinado tiempo, y  Airflow gestiona las dependencias facilitando la automatización del proceso.

## **Pregunta 4**
> Describa que pasos usted usaría para retirar campos vacíos y caracteres especiales (comas, saltos de línea, diagonales, pipelines, etc.) en un archivo CSV y depositarlo en un repositorio utilizando una herramienta de flujos analíticos que usted conozca (Alteryx, Tableau Prep, etc.). 
---

Power BI

- Con Power BI Desktop, voy a **obtener datos** y selecciono los archivos CSV.

- Abrir con Power Query en el editor trabajo sobre los datos para hacer limpieza.

- Filtro las filas para quitar valores vacíos, nulos, y de igual manera está la opción de quitar columnas para eliminar columnas sin datos relevantes.

- Con la opción **reemplazar valores** para quitar comas, saltos de línea, diagonales, pipes, etc. De igual forma, con la función replace o transformar, para limpiar o borrar caracteres no deseados.

- Revisión de duplicados, tipo de datos de las columnas, etc.

- Una vez limpio, publico el dataset en **Power BI Service** y desde ahí puede almacenarse en un workspace o conectarse a un repositorio como Azure SQL Database, OneDrive etc, para que otras personas puedan consumirlo, o bien exportar el CSV listo y subirlo a Git desde el repositorio local.

---

Con este flujo, con **Power BI** hacemos la limpieza y con **Git** podemos tener el control de versiones y compartir el archivo limpio con el equipo.